In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from nltk.corpus import stopwords  
from nltk.stem.lancaster import LancasterStemmer  
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('data.csv')

In [3]:
data.head(10)
data.shape

(850359, 23)

In [4]:
ask_data=data[data['subreddit']=='AskReddit']

In [5]:
ask_data=ask_data[ask_data['body']!='[deleted]']
ask_data.head(10)

,Unnamed: 0,gilded,author_flair_css_class,id,link_id,downs,subreddit,author,score,subreddit_id,...,body,retrieved_on,parent_id,controversiality,name,score_hidden,distinguished,archived,created_utc,removal_reason
21,21,0,NaN,o,t3_7k903,0,AskReddit,ermine,1,t5_2qh1i,...,Ok. Here's what you do. Get some speaker wire ...,1428217153,t3_7k903,0,t1_o,False,NaN,True,1229580158,NaN
26,26,0,NaN,t,t3_7k377,0,AskReddit,movzx,3,t5_2qh1i,...,lol yeah you're right! so hilarious!,1428217153,t1_c06vnj0,0,t1_t,False,NaN,True,1229580220,NaN
50,50,0,NaN,1h,t3_7k8xe,0,AskReddit,robrobrobot,5,t5_2qh1i,...,"Yeah, I've contemplated that quite a bit as we...",1428217153,t1_c06vwnq,0,t1_1h,False,NaN,True,1229580347,NaN
74,74,0,NaN,25,t3_7k92p,0,AskReddit,[deleted],1,t5_2qh1i,...,Let us plan this one :).......,1428217153,t1_c06vwn9,0,t1_25,False,NaN,True,1229580422,NaN
107,107,0,NaN,32,t3_7k7ff,0,AskReddit,kenlubin,1,t5_2qh1i,...,which?,1428217154,t3_7k7ff,0,t1_32,False,NaN,True,1229580531,NaN
127,127,0,NaN,3m,t3_7k7vs,0,AskReddit,krispykrackers,4,t5_2qh1i,...,♥qghy2♥\n\n\nHe's had a lot of reddit love tod...,1428217154,t3_7k7vs,0,t1_3m,False,NaN,True,1229580640,NaN
155,155,0,NaN,4g,t3_7k3z1,0,AskReddit,chibikiba,1,t5_2qh1i,...,We already have the shoes.,1428217154,t3_7k3z1,0,t1_4g,False,NaN,True,1229580768,NaN
166,166,0,NaN,4r,t3_7k90i,0,AskReddit,marydaze,-1,t5_2qh1i,...,"cool, cool",1428217154,t3_7k90i,0,t1_4r,False,NaN,True,1229580821,NaN
170,170,0,NaN,4v,t3_7k7vs,0,AskReddit,krispykrackers,3,t5_2qh1i,...,"You do know he abandoned us, right? I mean, I...",1428217154,t1_c06vvdm,0,t1_4v,False,NaN,True,1229580830,NaN
180,180,0,NaN,55,t3_7k377,0,AskReddit,ermine,2,t5_2qh1i,...,They don't consider you competent. You are sim...,1428217154,t3_7k377,0,t1_55,False,NaN,True,1229580843,NaN


In [6]:
ask_data.shape

(55006, 23)

In [7]:
set(stopwords.words('english') )

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

Pre-processing data

In [17]:
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)

punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

remove_n = lambda x: re.sub("\n", " ", x)

# Remove all non-ascii characters 
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)

#remove_deleted = lambda x: re.sub('deleted','', x)

# Apply all the lambda functions wrote previously through .map on the comments column
ask_data['body'] = ask_data['body'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

ask_data.head(10)

,Unnamed: 0,gilded,author_flair_css_class,id,link_id,downs,subreddit,author,score,subreddit_id,...,body,retrieved_on,parent_id,controversiality,name,score_hidden,distinguished,archived,created_utc,removal_reason
21,21,0,NaN,o,t3_7k903,0,AskReddit,ermine,1,t5_2qh1i,...,ok here s what you do get some speaker wire ...,1428217153,t3_7k903,0,t1_o,False,NaN,True,1229580158,NaN
26,26,0,NaN,t,t3_7k377,0,AskReddit,movzx,3,t5_2qh1i,...,lol yeah you re right so hilarious,1428217153,t1_c06vnj0,0,t1_t,False,NaN,True,1229580220,NaN
50,50,0,NaN,1h,t3_7k8xe,0,AskReddit,robrobrobot,5,t5_2qh1i,...,yeah i ve contemplated that quite a bit as we...,1428217153,t1_c06vwnq,0,t1_1h,False,NaN,True,1229580347,NaN
74,74,0,NaN,25,t3_7k92p,0,AskReddit,[deleted],1,t5_2qh1i,...,let us plan this one,1428217153,t1_c06vwn9,0,t1_25,False,NaN,True,1229580422,NaN
107,107,0,NaN,32,t3_7k7ff,0,AskReddit,kenlubin,1,t5_2qh1i,...,which,1428217154,t3_7k7ff,0,t1_32,False,NaN,True,1229580531,NaN
127,127,0,NaN,3m,t3_7k7vs,0,AskReddit,krispykrackers,4,t5_2qh1i,...,he s had a lot of reddit love today but...,1428217154,t3_7k7vs,0,t1_3m,False,NaN,True,1229580640,NaN
155,155,0,NaN,4g,t3_7k3z1,0,AskReddit,chibikiba,1,t5_2qh1i,...,we already have the shoes,1428217154,t3_7k3z1,0,t1_4g,False,NaN,True,1229580768,NaN
166,166,0,NaN,4r,t3_7k90i,0,AskReddit,marydaze,-1,t5_2qh1i,...,cool cool,1428217154,t3_7k90i,0,t1_4r,False,NaN,True,1229580821,NaN
170,170,0,NaN,4v,t3_7k7vs,0,AskReddit,krispykrackers,3,t5_2qh1i,...,you do know he abandoned us right i mean i...,1428217154,t1_c06vvdm,0,t1_4v,False,NaN,True,1229580830,NaN
180,180,0,NaN,55,t3_7k377,0,AskReddit,ermine,2,t5_2qh1i,...,they don t consider you competent you are sim...,1428217154,t3_7k377,0,t1_55,False,NaN,True,1229580843,NaN


In [9]:
# def model_diagnostics(model, pr=True):
#     """
#     Returns and prints the R-squared, RMSE and the MAE for a trained model
#     """
#     y_predicted = model.predict(X_test)
#     r2 = r2_score(y_test, y_predicted)
#     mse = mean_squared_error(y_test, y_predicted)
#     mae = mean_absolute_error(y_test, y_predicted)
#     if pr:
#         print(f"R-Sq: {r2:.4}")
#         print(f"RMSE: {np.sqrt(mse)}")
#         print(f"MAE: {mae}")
    
#     return [r2,np.sqrt(mse),mae]
# def plot_residuals(y_test, y_predicted):
#     """"
#     Plots the distribution for actual and predicted values of the target variable. Also plots the distribution for the residuals
#     """
#     fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharey=True)
#     sns.distplot(y_test, ax=ax0, kde = False)
#     ax0.set(xlabel='Test scores')
#     sns.distplot(y_predicted, ax=ax1, kde = False)
#     ax1.set(xlabel="Predicted scores")
#     plt.show()
#     fig, ax2 = plt.subplots()
#     sns.distplot((y_test-y_predicted), ax = ax2,kde = False)
#     ax2.set(xlabel="Residuals")
#     plt.show()
# def y_test_vs_y_predicted(y_test,y_predicted):
#     """
#     Produces a scatter plot for the actual and predicted values of the target variable
#     """
#     fig, ax = plt.subplots()
#     ax.scatter(y_test, y_predicted)
#     ax.set_xlabel("Test Scores")
#     ax.set_ylim([-75, 1400])
#     ax.set_ylabel("Predicted Scores")
#     plt.show()
# def get_feature_importance(model):
#     """
#     For fitted tree based models, get_feature_importance can be used to get the feature importance as a tidy output
#     """
#     X_non_text = pd.get_dummies(df[cat_cols])
#     features = numeric_cols + bool_cols + list(X_non_text.columns)
#     feature_importance = dict(zip(features, model.feature_importances_))
#     for name, importance in sorted(feature_importance.items(), key=lambda x: x[1], reverse=True):
#         print(f"{name:<30}: {importance:>6.2%}")
#         print(f"\nTotal importance: {sum(feature_importance.values()):.2%}")
#     return feature_importance

In [10]:
# bool_cols = ['score_hidden', 'archived']

# cat_cols = ['subreddit', 'subreddit_id', 'link_id']

# numeric_cols = ['gilded', 'controversiality', 'downs']

In [11]:
# lb = LabelBinarizer()
# cat = [lb.fit_transform(ask_data[col]) for col in cat_cols]
# bol = [ask_data[col].astype('int') for col in bool_cols]
# t = ask_data.loc[:, numeric_cols].values
# final = [t] + bol + cat
# y = ask_data.score.values
# x = np.column_stack(tuple(final))

In [12]:
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [13]:
# model_performance_dict = dict()

In [20]:
# Import packages for pre-processing
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel

# Import packages to split data and evaluate model performance
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, fbeta_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

# Import ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [22]:
def cv_tf_train_test(ask_data,score,vectorizer,ngram):

# Train/Test split
    X = ask_data.body
    y = ask_data[score]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Count Vectorizer/TF-IDF
    cv1 = vectorizer(ngram_range=(ngram), stop_words='english')
    
    X_train_cv1 = cv1.fit_transform(X_train)
    X_test_cv1  = cv1.transform(X_test)
         
# Initialize all model objects and fit the models on the training data
    lr = LogisticRegression()
    lr.fit(X_train_cv1, y_train)
    print('lr done')

    mnb = MultinomialNB()
    mnb.fit(X_train_cv1, y_train)
    print('mnb done')
    
    svm_model = LinearSVC()
    svm_model.fit(X_train_cv1, y_train)

    f1_score_data = {'F1 Score':[f1_score(lr.predict(X_test_cv1), y_test, average='micro'), f1_score(mnb.predict(X_test_cv1), y_test, average='micro'),
                                f1_score(svm_model.predict(X_test_cv1), y_test, average='micro')]} 
                          
    df_f1 = pd.DataFrame(f1_score_data, index=['Log Regression', 'MultinomialNB', 'SVM'])  

    return df_f1

In [23]:
df_tox_cv = cv_tf_train_test(ask_data, 'score', TfidfVectorizer, (1,1))
df_tox_cv.rename(columns={'F1 Score': 'F1 Score(upvotes)'}, inplace=True)
df_tox_cv

lr done
mnb done


,F1 Score(toxic)
Log Regression,0.298691
MultinomialNB,0.319598
SVM,0.248212


In [32]:
#data_tox_done = pd.concat([ask_data['body'], ask_data['score']], axis=0)
data_tox_done = ask_data.filter(['body','score'], axis=1)
data_tox_done.shape
data_tox_done.head(10)

,body,score
21,ok here s what you do get some speaker wire ...,1
26,lol yeah you re right so hilarious,3
50,yeah i ve contemplated that quite a bit as we...,5
74,let us plan this one,1
107,which,1
127,he s had a lot of reddit love today but...,4
155,we already have the shoes,1
166,cool cool,-1
170,you do know he abandoned us right i mean i...,3
180,they don t consider you competent you are sim...,2


In [33]:
X = data_tox_done.body
y = data_tox_done['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initiate a Tfidf vectorizer
tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

X_train_fit = tfv.fit_transform(X_train)
X_test_fit = tfv.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train_fit, y_train)

mnb.predict(X_test_fit)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [34]:
import pickle

In [43]:
def pickle_model(data, label):
    
    X = data.body
    y = data[label]

    # Initiate a Tfidf vectorizer
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

    X_vect = tfv.fit_transform(X)  

    with open(r"{}.pkl".format(label + '_vect'), "wb") as f:   
        pickle.dump(tfv, f)   
        
    mnb = MultinomialNB()
    mnb.fit(X_train_fit, y_train)

    with open(r"{}.pkl".format(label + '_model'), "wb") as f:  
        pickle.dump(mnb, f)        

In [44]:
datalist = [data_tox_done]
label = ['score']

for i,j in zip(datalist,label):
    pickle_model(i, j)

In [15]:
# baseline = DummyRegressor(strategy='mean')
# baseline.fit(X_train,y_train)
# model_performance_dict["Baseline"] = model_diagnostics(baseline)

R-Sq: -6.123e-05
RMSE: 13.048243614164234
MAE: 4.832002002230764


In [16]:
# linear = LinearRegression()
# linear.fit(X_train,y_train)
# model_performance_dict["Linear Regression"] = model_diagnostics(linear)

R-Sq: -7.002e+24
RMSE: 34525566084151.258
MAE: 1045349751212.6859
